In [4]:
jupyter nbconvert --to python *.ipynb

[NbConvertApp] Converting notebook Workspace_bash.ipynb to python
[NbConvertApp] Writing 522 bytes to Workspace_bash.py
[NbConvertApp] Converting notebook bam2gff.ipynb to python
[NbConvertApp] Writing 620 bytes to bam2gff.py
[NbConvertApp] Converting notebook gff2introns.ipynb to python
[NbConvertApp] Writing 3338 bytes to gff2introns.py


In [ ]:
# Prepare transcript annotations from Nanopore data

In [ ]:
# Mapping


In [ ]:
# generate transcript annotations
bash bam2gff.sh -b merged.L1.minimap2.sort.bam -o merged.L1.minimap2.sort.bam.gff -p /Users/kuroyanagi/pinfish

In [ ]:
# generate intron annotations
python gff2introns.py --GFF merged.L1.minimap2.sort.bam.gff

In [ ]:
# label gene names
intersectBed -s -loj -a collapsed_transcripts_N2_KH1668.gff\
-b /Users/kuroyanagi/expression/Caenorhabditis_elegans/WS271/Exon_caenorhabditis_elegans.PRJNA13758.WBPS14.canonical_geneset.gtf\
> collapsed_transcripts_N2_KH1668.int.gff

# filter and process transcripts in R

In [ ]:
# Machine learning

In [ ]:
# fastq to fasta
awk '(NR - 1) % 4 < 2' /Volumes/DATA_3/190926_DrKuroyanagi_sams_345/190926_DrKuroyanagi_sams_345/20190926_0543_MN17290_FAK89523_0ee12f5a/1d.fq |\
sed 's/@/>/' > /Volumes/DATA_3/190926_DrKuroyanagi_sams_345/190926_DrKuroyanagi_sams_345/20190926_0543_MN17290_FAK89523_0ee12f5a/1d.fasta

In [ ]:
# check barcodes


In [ ]:
# subset reads
fast5_subset --input $fast5_path --save_path $fast5_GUCAUCCC --read_id_list sams345_read_id_GUCAUCCC.txt
fast5_subset --input $fast5_path --save_path $fast5_GUCAUGGG --read_id_list sams345_read_id_GUCAUGGG.txt

In [ ]:
# unpack multifast5
multi_to_single_fast5 -i $fast5_GUCAUCCC -s $fast5_GUCAUCCC -t 10
multi_to_single_fast5 -i $fast5_GUCAUGGG -s $fast5_GUCAUGGG -t 10

In [ ]:
# Tombo annotate
tombo preprocess annotate_raw_with_fastqs --fast5-basedir $fast5_GUCAUCCC --fastq-filenames $fq_path --sequencing-summary-filenames $summary_path --processes 10 --overwrite --corrected-group RawGenomeCorrected_sams345
tombo preprocess annotate_raw_with_fastqs --fast5-basedir $fast5_GUCAUGGG --fastq-filenames $fq_path --sequencing-summary-filenames $summary_path --processes 10 --overwrite --corrected-group RawGenomeCorrected_sams345

In [ ]:
# Tombo mapping
tombo resquiggle --rna --processes 10 --num-most-common-errors 5 --overwrite --corrected-group RawGenomeCorrected_sams345 $fast5_GUCAUCCC sams-345_extension1kb.fa
tombo resquiggle --rna --processes 10 --num-most-common-errors 5 --overwrite --corrected-group RawGenomeCorrected_sams345 $fast5_GUCAUGGG sams-345_extension1kb.fa